In [1]:
# Import statements.
import numpy as np
import random as rand
import torch
import math
import matplotlib.pyplot as plt
from torch import nn
import torch.nn.functional as F
import torch.distributions as tdist
%matplotlib inline

In [2]:
# Generates network weights.
def generate_weights(starting_size, ending_size, weights_needed):
    difference = (starting_size - ending_size) / (weights_needed + 1)
    weights = []
    for i in range(weights_needed):
        weights.append(int(starting_size - (difference * (i+1))))
    return weights

In [3]:
# Link between two graph nodes.
class LINK:
    
    # Constructor.
    def __init__(self, start, end, reward, actions):
        self.start = start
        self.end = end
        self.reward = reward
        self.actions = actions
        self.distance = 0
        for i in range(len(start.state)):
            self.distance += (start.state[i] - end.state[i]) ** 2
        
    # Returns a copy of the action set.
    def copy_actions(self):
        return [a for a in self.actions]

# Graph Node.
class NODE:
    
    # Constructor.
    def __init__(self, state, discovery_time, op=None):
        self.state = state
        self.discovery_time = discovery_time
        self.forward_links = []
        self.backward_links = []
        self.op = op
        
    # Checks if the provided state is identical this this node.
    def is_same(self, state):
        if self.op is None:
            for i in range(len(self.state)):
                if self.state[i] != state[i]:
                    return False
            return True
        else:
            return self.op(self.state, state)
        
    # Creates a forward link between this node and the provided node.
    def construct_link(self, node, reward, actions):
        for link in self.forward_links:
            # Checks if a link between the nodes already exists.
            if link.end.is_same(node.state):
                # Updates the link to the new score and actions if they are better.
                if reward > link.reward:
                    link.reward = reward
                    link.actions = actions
                    return
        # If the link doesn't exist.
        link = LINK(self, node, reward, actions)
        self.forward_links.append(link)
        node.backward_links.append(link)

In [4]:
# Policy recommendor network.
class POLICY_NET(nn.Module):
    
    # Constructor.
    def __init__(self, input_size, output_size, layer_count, output_count, t_device):
        super().__init__()
        weights = generate_weights(input_size, output_size, layer_count)
        prev_weight = input_size
        self.t_device = t_device
        self.hidden_layers = []
        for w in weights:
            self.hidden_layers.append(nn.Linear(prev_weight, w).to(self.t_device))
            prev_weight = w
        self.output_layers = []
        for i in range(output_count):
            self.output_layers.append(nn.Linear(prev_weight, output_size).to(self.t_device))
        self.sigmoid = nn.Sigmoid()
        self.relu = F.relu
        self.softmax = nn.Softmax(dim=0)
        self.params = []
        for h in self.hidden_layers:
            self.params += list(h.parameters())
        for o in self.output_layers:
            self.params += list(o.parameters())
            
    # Forward propogate input.
    def forward(self, x):
        for hidden in self.hidden_layers:
            x = self.relu(hidden(x))
        outputs = []
        for out in self.output_layers:
            outputs.append(self.relu(out(x)))
        return outputs

In [5]:
# Exploratory agent.
class AGENT:
    
    # Constructor.
    def __init__(self, state_size, action_size, layer_count, step_size, learning_rate, gamma, t_device, s_device):
        self.state_size = state_size
        self.action_size = action_size
        self.layer_count = layer_count
        self.step_size = step_size
        self.learning_rate = learning_rate
        self.gamma = gamma
        self.t_device = t_device
        self.s_device = s_device
        self.age = 1
        self.node_levels = [[]]
        self.policy_map = POLICY_NET(state_size, action_size, layer_count, step_size, t_device)
        self.optimizer = torch.optim.Adam(self.policy_map.params, lr=learning_rate)
        self.loss_func = nn.CrossEntropyLoss()
        
    # Generates score biased training data.
    def get_score_training_data(self):
        pairings = []
        for level in self.node_levels:
            for node in level:
                if len(node.forward_links) > 0:
                    link = node.forward_links[0]
                    for l in node.forward_links:
                        if link.reward < l.reward:
                            link = l
                    if link.reward > 0:
                        pairings.append((torch.Tensor.float(torch.from_numpy(node.state)), link.copy_actions(), link.reward))
        pairings.sort(key = lambda x: x[2], reverse=True)
        pairings = pairings[:int(len(pairings) / 2)]
        rand.shuffle(pairings)
        inputs = []
        outputs = [[] for _ in range(self.step_size)]
        for p in pairings:
            inputs.append(p[0])
            for i in range(self.step_size):
                outputs[i].append(p[1][i])
        return inputs, outputs
    
    # Generates distance biased training data.
    def get_distance_training_data(self):
        pairings = []
        for level in self.node_levels:
            for node in level:
                if len(node.forward_links) > 0:
                    link = node.forward_links[0]
                    for l in node.forward_links:
                        if link.distance < l.distance:
                            link = l
                    if link.distance > 0: # Might want to change to cutting off on score not sure.
                        pairings.append((torch.Tensor.float(torch.from_numpy(node.state)), link.copy_actions(), link.distance))
        pairings.sort(key = lambda x: x[2], reverse=True)
        pairings = pairings[:int(len(pairings) / 2)]
        rand.shuffle(pairings)
        inputs = []
        outputs = [[] for _ in range(self.step_size)]
        for p in pairings:
            inputs.append(p[0])
            for i in range(self.step_size):
                outputs[i].append(p[1][i])
        return inputs, outputs
    
    # Generates distance and score biased training data.
    def get_composite_training_data(self):
        pairings = []
        for level in self.node_levels:
            for node in level:
                if len(node.forward_links) > 0:
                    link = node.forward_links[0]
                    for l in node.forward_links:
                        if (link.reward + 0.01) * link.distance < (l.reward + 0.01) * l.distance:
                            link = l
                    if link.reward > 0:
                        pairings.append((torch.Tensor.float(torch.from_numpy(node.state)), link.copy_actions(), (link.reward + 0.01) * link.distance))
        pairings.sort(key = lambda x: x[2], reverse=True)
        pairings = pairings[:int(len(pairings) / 2)]
        rand.shuffle(pairings)
        inputs = []
        outputs = [[] for _ in range(self.step_size)]
        for p in pairings:
            inputs.append(p[0])
            for i in range(self.step_size):
                outputs[i].append(p[1][i])
        return inputs, outputs
    
    # Train policy network.
    def train_policy_network(self, batch_size=1024, epochs=50):
        #inputs, outputs = self.get_score_training_data()
        #inputs, outputs = self.get_distance_training_data()
        inputs, outputs = self.get_composite_training_data()
        batches = []
        position = 0
        batch = ([],[[] for _ in range(self.step_size)])
        while position < len(inputs):
            batch[0].append(inputs[position])
            for i in range(len(batch[1])):
                batch[1][i].append(outputs[i][position])
            position += 1
            if len(batch[0]) >= batch_size:
                batches.append(batch)
                batch = ([],[[] for _ in range(self.step_size)])
        if len(batch) > 0:
            batches.append(batch)
        for e in range(epochs):
            for i in range(len(batches)):
                inputs = torch.stack(batches[i][0])
                outputs = [torch.Tensor(o).long() for o in batches[i][1]]
                out = self.policy_map(inputs)
                loss = None
                for j in range(len(outputs)):
                    if loss is None:
                        loss = self.loss_func(out[j], outputs[j])
                    else:
                        loss += self.loss_func(out[j], outputs[j])
                print('\rEPOCH {}/{} | BATCH {}/{} | CURRENT BATCH COUNT {} | LOSS {}'.format(e+1, epochs, i+1, len(batches), len(batches[i][0]), loss.detach().cpu().numpy()), end='')
                loss.backward()
                self.optimizer.step()
            
        
    # Plays a game.
    def play_game(self, env, render=False):
        done = False
        previous_node = None
        action = 0
        score = 0
        inner_score = 0 # Score inside inner steps.
        overall_step = 0
        step = 0
        depth = 0
        first_step = True
        lives = 4
        action_queue = None
        env.reset()
        prev_root_count = len(self.node_levels[0])
        while not done:
            if first_step:
                observation, reward, done, info = env.step(action)
                state = observation
                found = False
                for n in self.node_levels[0]:
                    if n.is_same(state):
                        previous_node = n
                        found = True
                        break
                if not found:
                    previous_node = NODE(state, self.age)
                    self.node_levels[0].append(previous_node)
                tensor = torch.Tensor.float(torch.from_numpy(state))
                dists = self.policy_map(tensor)
                action_queue = []
                for d in dists:
                    if rand.uniform(0,1) > self.gamma or min(d) < 0 or sum(d) == 0:
                        action_queue.append(rand.randint(0, self.action_size - 1))
                    else:
                        distribution = torch.distributions.categorical.Categorical(d)
                        action_queue.append(int(distribution.sample()))
                first_step = False
            else:
                action = action_queue[step]
                observation, reward, done, info = env.step(action)
                score += reward
                inner_score += reward
                state = observation
                step += 1
                if step == self.step_size:
                    depth += 1
                    current_node = None
                    if depth < len(self.node_levels):
                        found = False
                        for n in self.node_levels[depth]:
                            if n.is_same(state):
                                current_node = n
                                found = True
                                break
                        if not found:
                            current_node = NODE(state, self.age)
                            self.node_levels[depth].append(current_node)
                    else:
                        current_node = NODE(state, self.age)
                        self.node_levels.append([current_node])
                    previous_node.construct_link(current_node, inner_score, action_queue)
                    step = 0
                    inner_score = 0
                    tensor = torch.Tensor.float(torch.from_numpy(state))
                    dists = self.policy_map(tensor)
                    action_queue = []
                    try_rand = rand.uniform(0,1)
                    for d in dists:
                        if try_rand > self.gamma or min(d) < 0 or sum(d) == 0:
                            action_queue.append(rand.randint(0, self.action_size - 1))
                        else:
                            distribution = torch.distributions.categorical.Categorical(d)
                            action_queue.append(int(distribution.sample()))
                    previous_node = current_node
            print('\rSTEP {} | SCORE {} | DEPTH {} | ROOTS {} | AGE {} | RATIO {:0.4f} | ADDED {}\t\t'.format(overall_step, score, depth, len(self.node_levels[0]), self.age, len(self.node_levels[0])/self.age, len(self.node_levels[0])-prev_root_count), end = '')
            if render:
                env.render()
            if info['ale.lives'] != lives or done:
                lives = info['ale.lives']
                previous_node = None
                action = 0
                inner_score = 0 # Score inside inner steps.
                step = 0
                depth = 0
                first_step = True
                action_queue = None
            overall_step += 1
        self.age += 1
        

In [6]:
agent = AGENT(128, 14, 10, 5, 0.01, 0.9, torch.device('cpu'), torch.device('cpu'))

In [7]:
import gym
env = gym.make('KungFuMaster-ram-v0')

In [ ]:
while True:
    agent.play_game(env, True)
    print()
    if agent.age % 10 == 0:
        agent.train_policy_network()
        print()

STEP 38 | SCORE 0.0 | DEPTH 7 | ROOTS 1 | AGE 1 | RATIO 1.0000 | ADDED 1		

c:\users\max9613\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in ubyte_scalars
  if sys.path[0] == '':


STEP 1294 | SCORE 600.0 | DEPTH 46 | ROOTS 4 | AGE 1 | RATIO 4.0000 | ADDED 4		
STEP 1454 | SCORE 600.0 | DEPTH 63 | ROOTS 8 | AGE 2 | RATIO 4.0000 | ADDED 4		
STEP 1132 | SCORE 400.0 | DEPTH 64 | ROOTS 12 | AGE 3 | RATIO 4.0000 | ADDED 4		
STEP 809 | SCORE 100.0 | DEPTH 30 | ROOTS 16 | AGE 4 | RATIO 4.0000 | ADDED 4		
STEP 1442 | SCORE 800.0 | DEPTH 76 | ROOTS 20 | AGE 5 | RATIO 4.0000 | ADDED 4		
STEP 1287 | SCORE 400.0 | DEPTH 61 | ROOTS 24 | AGE 6 | RATIO 4.0000 | ADDED 4		
STEP 1066 | SCORE 600.0 | DEPTH 31 | ROOTS 27 | AGE 7 | RATIO 3.8571 | ADDED 3		
STEP 941 | SCORE 700.0 | DEPTH 39 | ROOTS 31 | AGE 8 | RATIO 3.8750 | ADDED 4		
STEP 1348 | SCORE 300.0 | DEPTH 65 | ROOTS 35 | AGE 9 | RATIO 3.8889 | ADDED 4		
EPOCH 50/50 | BATCH 1/1 | CURRENT BATCH COUNT 13 | LOSS 10.241255760192871
STEP 1730 | SCORE 2300.0 | DEPTH 107 | ROOTS 39 | AGE 10 | RATIO 3.9000 | ADDED 4		
STEP 1607 | SCORE 2800.0 | DEPTH 108 | ROOTS 43 | AGE 11 | RATIO 3.9091 | ADDED 4		
STEP 1605 | SCORE 2000.0 | DEPTH

STEP 1469 | SCORE 1900.0 | DEPTH 56 | ROOTS 345 | AGE 90 | RATIO 3.8333 | ADDED 4		
STEP 1925 | SCORE 4300.0 | DEPTH 87 | ROOTS 349 | AGE 91 | RATIO 3.8352 | ADDED 4			
STEP 1694 | SCORE 2000.0 | DEPTH 91 | ROOTS 353 | AGE 92 | RATIO 3.8370 | ADDED 4		
STEP 1811 | SCORE 3100.0 | DEPTH 84 | ROOTS 357 | AGE 93 | RATIO 3.8387 | ADDED 4			
STEP 1964 | SCORE 2600.0 | DEPTH 101 | ROOTS 361 | AGE 94 | RATIO 3.8404 | ADDED 4		
STEP 1752 | SCORE 3300.0 | DEPTH 110 | ROOTS 365 | AGE 95 | RATIO 3.8421 | ADDED 4		
STEP 1531 | SCORE 1100.0 | DEPTH 70 | ROOTS 369 | AGE 96 | RATIO 3.8438 | ADDED 4		
STEP 1522 | SCORE 2000.0 | DEPTH 89 | ROOTS 373 | AGE 97 | RATIO 3.8454 | ADDED 4		
STEP 2071 | SCORE 3300.0 | DEPTH 116 | ROOTS 377 | AGE 98 | RATIO 3.8469 | ADDED 4		
STEP 1490 | SCORE 2800.0 | DEPTH 89 | ROOTS 381 | AGE 99 | RATIO 3.8485 | ADDED 4		
EPOCH 50/50 | BATCH 1/1 | CURRENT BATCH COUNT 577 | LOSS 8.653231620788574
STEP 1094 | SCORE 400.0 | DEPTH 39 | ROOTS 384 | AGE 100 | RATIO 3.8400 | ADDED 

STEP 1941 | SCORE 2800.0 | DEPTH 68 | ROOTS 673 | AGE 179 | RATIO 3.7598 | ADDED 3			
EPOCH 50/50 | BATCH 1/1 | CURRENT BATCH COUNT 1018 | LOSS 9.141818046569824
STEP 1684 | SCORE 1200.0 | DEPTH 60 | ROOTS 676 | AGE 180 | RATIO 3.7556 | ADDED 3		
STEP 1891 | SCORE 2200.0 | DEPTH 68 | ROOTS 679 | AGE 181 | RATIO 3.7514 | ADDED 3		
STEP 1595 | SCORE 2500.0 | DEPTH 67 | ROOTS 682 | AGE 182 | RATIO 3.7473 | ADDED 3		
STEP 1710 | SCORE 1600.0 | DEPTH 115 | ROOTS 686 | AGE 183 | RATIO 3.7486 | ADDED 4		
STEP 1545 | SCORE 1300.0 | DEPTH 44 | ROOTS 689 | AGE 184 | RATIO 3.7446 | ADDED 3		
STEP 1756 | SCORE 1900.0 | DEPTH 70 | ROOTS 692 | AGE 185 | RATIO 3.7405 | ADDED 3		
STEP 1836 | SCORE 3400.0 | DEPTH 79 | ROOTS 695 | AGE 186 | RATIO 3.7366 | ADDED 3			
STEP 2122 | SCORE 4300.0 | DEPTH 109 | ROOTS 698 | AGE 187 | RATIO 3.7326 | ADDED 3		
STEP 1736 | SCORE 2000.0 | DEPTH 82 | ROOTS 701 | AGE 188 | RATIO 3.7287 | ADDED 3		
STEP 1636 | SCORE 1200.0 | DEPTH 79 | ROOTS 704 | AGE 189 | RATIO 3.72

...

STEP 1449 | SCORE 1500.0 | DEPTH 67 | ROOTS 3569 | AGE 1047 | RATIO 3.4088 | ADDED 3		
STEP 1335 | SCORE 2400.0 | DEPTH 77 | ROOTS 3572 | AGE 1048 | RATIO 3.4084 | ADDED 3		
STEP 1652 | SCORE 1400.0 | DEPTH 57 | ROOTS 3575 | AGE 1049 | RATIO 3.4080 | ADDED 3		
EPOCH 50/50 | BATCH 8/8 | CURRENT BATCH COUNT 521 | LOSS 11.5815238952636727
STEP 1542 | SCORE 1700.0 | DEPTH 66 | ROOTS 3578 | AGE 1050 | RATIO 3.4076 | ADDED 3			
STEP 2091 | SCORE 4300.0 | DEPTH 119 | ROOTS 3581 | AGE 1051 | RATIO 3.4072 | ADDED 3		
STEP 1596 | SCORE 2600.0 | DEPTH 71 | ROOTS 3585 | AGE 1052 | RATIO 3.4078 | ADDED 4		
STEP 1392 | SCORE 1500.0 | DEPTH 72 | ROOTS 3588 | AGE 1053 | RATIO 3.4074 | ADDED 3		
STEP 1593 | SCORE 1600.0 | DEPTH 42 | ROOTS 3591 | AGE 1054 | RATIO 3.4070 | ADDED 3		
STEP 1554 | SCORE 2000.0 | DEPTH 86 | ROOTS 3594 | AGE 1055 | RATIO 3.4066 | ADDED 3		
STEP 1654 | SCORE 3300.0 | DEPTH 85 | ROOTS 3597 | AGE 1056 | RATIO 3.4062 | ADDED 3			
STEP 1626 | SCORE 2000.0 | DEPTH 65 | ROOTS 3600 |

STEP 1806 | SCORE 2400.0 | DEPTH 60 | ROOTS 3845 | AGE 1133 | RATIO 3.3936 | ADDED 3			
STEP 1298 | SCORE 1000.0 | DEPTH 80 | ROOTS 3848 | AGE 1134 | RATIO 3.3933 | ADDED 3		
STEP 1666 | SCORE 2100.0 | DEPTH 128 | ROOTS 3851 | AGE 1135 | RATIO 3.3930 | ADDED 3		
STEP 1340 | SCORE 1000.0 | DEPTH 73 | ROOTS 3854 | AGE 1136 | RATIO 3.3926 | ADDED 3		
STEP 1906 | SCORE 2000.0 | DEPTH 94 | ROOTS 3857 | AGE 1137 | RATIO 3.3923 | ADDED 3		
STEP 1745 | SCORE 2500.0 | DEPTH 83 | ROOTS 3860 | AGE 1138 | RATIO 3.3919 | ADDED 3		
STEP 1703 | SCORE 3200.0 | DEPTH 74 | ROOTS 3863 | AGE 1139 | RATIO 3.3916 | ADDED 3		
EPOCH 50/50 | BATCH 8/8 | CURRENT BATCH COUNT 995 | LOSS 11.6765336990356455
STEP 1596 | SCORE 1400.0 | DEPTH 47 | ROOTS 3866 | AGE 1140 | RATIO 3.3912 | ADDED 3		
STEP 1907 | SCORE 2500.0 | DEPTH 74 | ROOTS 3870 | AGE 1141 | RATIO 3.3918 | ADDED 4		
STEP 1454 | SCORE 1600.0 | DEPTH 96 | ROOTS 3873 | AGE 1142 | RATIO 3.3914 | ADDED 3		
STEP 1633 | SCORE 1400.0 | DEPTH 62 | ROOTS 3876 | 

STEP 1538 | SCORE 1300.0 | DEPTH 75 | ROOTS 4119 | AGE 1219 | RATIO 3.3790 | ADDED 3		
EPOCH 50/50 | BATCH 9/9 | CURRENT BATCH COUNT 396 | LOSS 11.2180538177490235
STEP 1566 | SCORE 1900.0 | DEPTH 47 | ROOTS 4123 | AGE 1220 | RATIO 3.3795 | ADDED 4		
STEP 1456 | SCORE 1700.0 | DEPTH 132 | ROOTS 4126 | AGE 1221 | RATIO 3.3792 | ADDED 3		
STEP 1492 | SCORE 1500.0 | DEPTH 103 | ROOTS 4129 | AGE 1222 | RATIO 3.3789 | ADDED 3		
STEP 1750 | SCORE 2600.0 | DEPTH 123 | ROOTS 4132 | AGE 1223 | RATIO 3.3786 | ADDED 3		
STEP 1906 | SCORE 1800.0 | DEPTH 40 | ROOTS 4135 | AGE 1224 | RATIO 3.3783 | ADDED 3			
STEP 1273 | SCORE 1200.0 | DEPTH 74 | ROOTS 4138 | AGE 1225 | RATIO 3.3780 | ADDED 3		
STEP 1344 | SCORE 1300.0 | DEPTH 79 | ROOTS 4141 | AGE 1226 | RATIO 3.3777 | ADDED 3		
STEP 1311 | SCORE 900.0 | DEPTH 36 | ROOTS 4144 | AGE 1227 | RATIO 3.3773 | ADDED 3		
STEP 1552 | SCORE 1600.0 | DEPTH 64 | ROOTS 4148 | AGE 1228 | RATIO 3.3779 | ADDED 4		
STEP 1916 | SCORE 2100.0 | DEPTH 71 | ROOTS 4151 |

STEP 1852 | SCORE 2500.0 | DEPTH 66 | ROOTS 4390 | AGE 1304 | RATIO 3.3666 | ADDED 3			
STEP 1551 | SCORE 2400.0 | DEPTH 62 | ROOTS 4393 | AGE 1305 | RATIO 3.3663 | ADDED 3		
STEP 1439 | SCORE 2000.0 | DEPTH 44 | ROOTS 4396 | AGE 1306 | RATIO 3.3660 | ADDED 3		
STEP 1610 | SCORE 1400.0 | DEPTH 77 | ROOTS 4400 | AGE 1307 | RATIO 3.3665 | ADDED 4		
STEP 1268 | SCORE 1800.0 | DEPTH 33 | ROOTS 4403 | AGE 1308 | RATIO 3.3662 | ADDED 3		
STEP 1254 | SCORE 1200.0 | DEPTH 64 | ROOTS 4406 | AGE 1309 | RATIO 3.3659 | ADDED 3		
EPOCH 50/50 | BATCH 9/9 | CURRENT BATCH COUNT 874 | LOSS 11.9653387069702154
STEP 1527 | SCORE 2000.0 | DEPTH 72 | ROOTS 4409 | AGE 1310 | RATIO 3.3656 | ADDED 3		
STEP 1299 | SCORE 1100.0 | DEPTH 69 | ROOTS 4412 | AGE 1311 | RATIO 3.3654 | ADDED 3		
STEP 1598 | SCORE 1800.0 | DEPTH 60 | ROOTS 4415 | AGE 1312 | RATIO 3.3651 | ADDED 3		
STEP 1933 | SCORE 3600.0 | DEPTH 65 | ROOTS 4418 | AGE 1313 | RATIO 3.3648 | ADDED 3			
STEP 1624 | SCORE 1300.0 | DEPTH 87 | ROOTS 4421 | 

STEP 1976 | SCORE 3300.0 | DEPTH 78 | ROOTS 4664 | AGE 1390 | RATIO 3.3554 | ADDED 3			
STEP 1241 | SCORE 1200.0 | DEPTH 65 | ROOTS 4667 | AGE 1391 | RATIO 3.3551 | ADDED 3		
STEP 1380 | SCORE 1600.0 | DEPTH 79 | ROOTS 4670 | AGE 1392 | RATIO 3.3549 | ADDED 3		
STEP 1515 | SCORE 3100.0 | DEPTH 68 | ROOTS 4673 | AGE 1393 | RATIO 3.3546 | ADDED 3		
STEP 1789 | SCORE 3000.0 | DEPTH 85 | ROOTS 4676 | AGE 1394 | RATIO 3.3544 | ADDED 3		
STEP 1466 | SCORE 1400.0 | DEPTH 73 | ROOTS 4679 | AGE 1395 | RATIO 3.3541 | ADDED 3		
STEP 1740 | SCORE 1600.0 | DEPTH 51 | ROOTS 4683 | AGE 1396 | RATIO 3.3546 | ADDED 4			
STEP 1409 | SCORE 1000.0 | DEPTH 70 | ROOTS 4686 | AGE 1397 | RATIO 3.3543 | ADDED 3		
STEP 1472 | SCORE 1500.0 | DEPTH 55 | ROOTS 4689 | AGE 1398 | RATIO 3.3541 | ADDED 3		
STEP 1307 | SCORE 1400.0 | DEPTH 57 | ROOTS 4692 | AGE 1399 | RATIO 3.3538 | ADDED 3		
EPOCH 50/50 | BATCH 10/10 | CURRENT BATCH COUNT 303 | LOSS 11.604986190795898
STEP 1253 | SCORE 1100.0 | DEPTH 49 | ROOTS 4695 |